In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file, get_centralized_market
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
# load Omnipool
print('loading omnipool...')
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=False)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("archive/")
    
# pprint(tokens)
init_state = OmnipoolState(
    tokens=tokens,
    lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list},
    asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list},
    preferred_stablecoin='USDT',
)

loading omnipool...


In [5]:
# load centralized markets
print('loading centralized markets...')
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"

with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)

for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])
    d['tkn_pair'] = (asset_map[d['tkn_ids'][0]], asset_map[d['tkn_ids'][1]])

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}

cex_dict = {}
for exchange in ('kraken', 'binance'):
    if get_live_markets:
        cex_dict[exchange] = get_centralized_market(
            config=cfg, 
            exchange_name=exchange, 
            trade_fee=cex_fees[exchange],
            archive=False
        )
    else:
        cex_dict[exchange] = CentralizedMarket(
            order_book=get_orderbooks_from_file("archive/")[exchange],
            unique_id=exchange,
            trade_fee=cex_fees[exchange]
        )

loading centralized markets...


In [6]:
print('calculating arbitrage...')
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = init_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex_dict1, cfg)

# pprint(all_swaps1)
# pprint(len(all_swaps1))

cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = init_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex_dict2, cfg)
# print("strategy 2")
# pprint(all_swaps2)
# pprint(len(all_swaps2))
# print(cfg)

calculating arbitrage...


In [7]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
arb.execute_arb(state2, cex_dict2, agent2, all_swaps2)

asset_map = {
    'WETH': 'ETH',
    'XETH': 'ETH',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'USDT001': 'USD',
    'DAI001': 'USD',
    'WETH001': 'ETH',
    'WBTC001': 'BTC',
    'iBTC': 'BTC',
    'XBT': 'BTC',
}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Binance spot")
print(cex_dict1['binance'].value_assets(profit1, asset_map))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Binance spot")
print(cex_dict2['binance'].value_assets(profit2, asset_map))

# try rerouting to see if we can squeeze some more profit out of it
agent3 = init_agent.copy()
state3 = init_state.copy()
cex_dict3 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
all_swaps3 = arb.combine_swaps(
    dex=state3,
    cex=cex_dict3,
    agent=agent3,
    all_swaps=all_swaps2,
    asset_map=asset_map
)
arb.execute_arb(state3, cex_dict3, agent3, all_swaps3)
profit3 = arb.calculate_profit(init_agent, agent3, asset_map)

print("strategy 3 profit")
print(profit3)
print("strategy 3 profit at Binance spot")
print(cex_dict3['binance'].value_assets(profit3, asset_map))


strategy 1 profit
{'4-Pool': 0, 'HDX': 0.0, 'USD': 50.34396779537201, 'ETH': 0.0, 'GLMR': 0.0, 'BTC': 6.9141387939453125e-06, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 1 profit at Binance spot
50.59956338644028
strategy 2 profit
{'4-Pool': 0, 'HDX': 0.0, 'USD': 60.01203787326813, 'ETH': 0, 'GLMR': 0.0, 'BTC': 0.0, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 2 profit at Binance spot
60.01203787326813
strategy 3 profit
{'4-Pool': 0, 'HDX': 0.0, 'USD': 67.78563404083252, 'ETH': 0, 'GLMR': 0.0, 'BTC': 2.384185791015625e-07, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 3 profit at Binance spot
67.79444768190383
